# FloPy

### Quick demo on how FloPy handles external files for arrays

In [1]:
import os 
import sys
import shutil
import numpy as np

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.5.2 | packaged by conda-forge | (default, Jan 12 2017, 05:36:40) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.54)]
numpy version: 1.11.3
flopy version: 3.2.6


In [2]:
# make a model
nlay,nrow,ncol = 10,20,5
model_ws = os.path.join("data","external_demo")

if os.path.exists(model_ws):
    shutil.rmtree(model_ws)

# the place for all of your hand made and costly model inputs    
array_dir = os.path.join("data","array_dir")
if os.path.exists(array_dir):
    shutil.rmtree(array_dir)    
os.mkdir(array_dir)    
    
ml = flopy.modflow.Modflow(model_ws=model_ws)
dis = flopy.modflow.ModflowDis(ml,nlay=nlay,nrow=nrow,ncol=ncol,steady=False,nper=2)

make an ``hk`` and ```vka``` array.  We'll save ```hk``` to files - pretent that you spent months making this important model property.  Then make an ```lpf```

In [3]:
hk = np.zeros((nlay,nrow,ncol)) + 5.0
vka = np.zeros_like(hk)
fnames = []
for i,h in enumerate(hk):
    fname = os.path.join(array_dir,"hk_{0}.ref".format(i+1))
    fnames.append(fname)
    np.savetxt(fname,h)
    vka[i] = i+1
lpf = flopy.modflow.ModflowLpf(ml,hk=fnames,vka=vka)

Let's also have some recharge with mixed args as well.  Pretend the recharge in the second stress period is very important and precise

In [4]:
warmup_recharge = np.ones((nrow,ncol))
important_recharge = np.random.random((nrow,ncol))
fname = os.path.join(array_dir,"important_recharge.ref")
np.savetxt(fname,important_recharge)
rch = flopy.modflow.ModflowRch(ml,rech={0:warmup_recharge,1:fname})

In [5]:
ml.write_input()

Let's look at the files that were created

In [6]:
print("model_ws:",ml.model_ws)
print('\n'.join(os.listdir(ml.model_ws)))

model_ws: data/external_demo
hk_1.ref
hk_10.ref
hk_2.ref
hk_3.ref
hk_4.ref
hk_5.ref
hk_6.ref
hk_7.ref
hk_8.ref
hk_9.ref
important_recharge.ref
modflowtest.dis
modflowtest.lpf
modflowtest.nam
modflowtest.rch


We see that a copy of the ``hk`` files as well as the important recharge file were made in the ```model_ws```.Let's looks at the ```lpf``` file

In [7]:
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()[:20]

['# LPF package for  MODFLOW-2005, generated by Flopy.\n',
 '         0    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                        hk_1.ref               1     (FREE) -1 hk layer 1                    \n',
 'INTERNAL               1   (5E15.6) -1 #vka layer 1                   \n',
 '   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00\n',
 '   1.000000E+00   1.000000E+00   1.000000E+00 

We see that the ```open/close``` approach was used - this is because ``ml.array_free_format`` is ``True``.  Notice that ```vka``` is written internally

In [8]:
ml.array_free_format

True

Now change ```model_ws```

In [9]:
print(ml.model_ws)
ml.model_ws = os.path.join("data","new_external_demo_dir")

data/external_demo

changing model workspace...
   data/new_external_demo_dir


Now when we call ``write_input()``, a copy of external files are made in the current ```model_ws```

In [10]:
ml.write_input()

In [11]:
# list the files in model_ws that have 'hk' in the name
print('\n'.join([name for name in os.listdir(ml.model_ws) if "hk" in name or "impor" in name]))

hk_1.ref
hk_10.ref
hk_2.ref
hk_3.ref
hk_4.ref
hk_5.ref
hk_6.ref
hk_7.ref
hk_8.ref
hk_9.ref
important_recharge.ref


Now we see that the external files were copied to the new ```model_ws```

### Using ```external_path```

It is sometimes useful when first building a model to write the model arrays as external files for processing and parameter estimation.  The ```model``` attribute ```external_path``` triggers this behavior

In [12]:
# make a model - same code as before except for the model constructor
nlay,nrow,ncol = 10,20,5
model_ws = os.path.join("data","external_demo")

if os.path.exists(model_ws):
    shutil.rmtree(model_ws)

# the place for all of your hand made and costly model inputs    
array_dir = os.path.join("data","array_dir")
if os.path.exists(array_dir):
    shutil.rmtree(array_dir)    
os.mkdir(array_dir)    

# lets make an external path relative to the model_ws
ml = flopy.modflow.Modflow(model_ws=model_ws, external_path="ref")
dis = flopy.modflow.ModflowDis(ml,nlay=nlay,nrow=nrow,ncol=ncol,steady=False,nper=2)

hk = np.zeros((nlay,nrow,ncol)) + 5.0
vka = np.zeros_like(hk)
fnames = []
for i,h in enumerate(hk):
    fname = os.path.join(array_dir,"hk_{0}.ref".format(i+1))
    fnames.append(fname)
    np.savetxt(fname,h)
    vka[i] = i+1
lpf = flopy.modflow.ModflowLpf(ml,hk=fnames,vka=vka) 

warmup_recharge = np.ones((nrow,ncol))
important_recharge = np.random.random((nrow,ncol))
fname = os.path.join(array_dir,"important_recharge.ref")
np.savetxt(fname,important_recharge)
rch = flopy.modflow.ModflowRch(ml,rech={0:warmup_recharge,1:fname})

We can see that the model constructor created both ```model_ws``` and ```external_path``` which is _relative to the model_ws_

In [13]:
os.listdir(ml.model_ws)

['ref']

Now, when we call ```write_input()```, any array properties that were specified as ```np.ndarray``` will be written externally.  If a scalar was passed as the argument, the value remains internal to the model input files

In [14]:
ml.write_input()
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()[:20]

Util2d:delr: resetting 'how' to external
Util2d:delc: resetting 'how' to external
Util2d:model_top: resetting 'how' to external
Util2d:botm layer 1: resetting 'how' to external
Util2d:botm layer 2: resetting 'how' to external
Util2d:botm layer 3: resetting 'how' to external
Util2d:botm layer 4: resetting 'how' to external
Util2d:botm layer 5: resetting 'how' to external
Util2d:botm layer 6: resetting 'how' to external
Util2d:botm layer 7: resetting 'how' to external
Util2d:botm layer 8: resetting 'how' to external
Util2d:botm layer 9: resetting 'how' to external
Util2d:botm layer 10: resetting 'how' to external
Util2d:ss layer 1: resetting 'how' to external
Util2d:ss layer 2: resetting 'how' to external
Util2d:ss layer 3: resetting 'how' to external
Util2d:ss layer 4: resetting 'how' to external
Util2d:ss layer 5: resetting 'how' to external
Util2d:ss layer 6: resetting 'how' to external
Util2d:ss layer 7: resetting 'how' to external
Util2d:ss layer 8: resetting 'how' to external
Util2

['# LPF package for  MODFLOW-2005, generated by Flopy.\n',
 '         0    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                    ref/hk_1.ref               1     (FREE) -1 hk layer 1                    \n',
 'OPEN/CLOSE             ref/vka_layer_1.ref               1   (5E15.6) -1 vka layer 1                   \n',
 'OPEN/CLOSE              ref/ss_layer_1.ref               1   (5E15.6) -1 ss layer 1            

Now, ```vka``` was also written externally, but not the storage properties.Let's verify the contents of the external path directory. We see our hard-fought ```hk``` and ```important_recharge``` arrays, as well as the ``vka`` arrays.

In [15]:
ml.lpf.ss.how = "internal"
ml.write_input()
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()[:20]

Util2d:delr: resetting 'how' to external
Util2d:delc: resetting 'how' to external
Util2d:model_top: resetting 'how' to external
Util2d:botm layer 1: resetting 'how' to external
Util2d:botm layer 2: resetting 'how' to external
Util2d:botm layer 3: resetting 'how' to external
Util2d:botm layer 4: resetting 'how' to external
Util2d:botm layer 5: resetting 'how' to external
Util2d:botm layer 6: resetting 'how' to external
Util2d:botm layer 7: resetting 'how' to external
Util2d:botm layer 8: resetting 'how' to external
Util2d:botm layer 9: resetting 'how' to external
Util2d:botm layer 10: resetting 'how' to external
Util2d:ss layer 1: resetting 'how' to external
Util2d:ss layer 2: resetting 'how' to external
Util2d:ss layer 3: resetting 'how' to external
Util2d:ss layer 4: resetting 'how' to external
Util2d:ss layer 5: resetting 'how' to external
Util2d:ss layer 6: resetting 'how' to external
Util2d:ss layer 7: resetting 'how' to external
Util2d:ss layer 8: resetting 'how' to external
Util2

['# LPF package for  MODFLOW-2005, generated by Flopy.\n',
 '         0    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 'OPEN/CLOSE                    ref/hk_1.ref               1     (FREE) -1 hk layer 1                    \n',
 'OPEN/CLOSE             ref/vka_layer_1.ref               1   (5E15.6) -1 vka layer 1                   \n',
 'OPEN/CLOSE              ref/ss_layer_1.ref               1   (5E15.6) -1 ss layer 1            

In [16]:
print('\n'.join(os.listdir(os.path.join(ml.model_ws,ml.external_path))))

botm_layer_1.ref
botm_layer_10.ref
botm_layer_2.ref
botm_layer_3.ref
botm_layer_4.ref
botm_layer_5.ref
botm_layer_6.ref
botm_layer_7.ref
botm_layer_8.ref
botm_layer_9.ref
delc.ref
delr.ref
hk_1.ref
hk_10.ref
hk_2.ref
hk_3.ref
hk_4.ref
hk_5.ref
hk_6.ref
hk_7.ref
hk_8.ref
hk_9.ref
important_recharge.ref
model_top.ref
rech_0.ref
ss_layer_1.ref
ss_layer_10.ref
ss_layer_2.ref
ss_layer_3.ref
ss_layer_4.ref
ss_layer_5.ref
ss_layer_6.ref
ss_layer_7.ref
ss_layer_8.ref
ss_layer_9.ref
vka_layer_1.ref
vka_layer_10.ref
vka_layer_2.ref
vka_layer_3.ref
vka_layer_4.ref
vka_layer_5.ref
vka_layer_6.ref
vka_layer_7.ref
vka_layer_8.ref
vka_layer_9.ref


### Fixed format

All of this behavior also works for fixed-format type models (really, really old models - I mean OLD!)

In [17]:
# make a model - same code as before except for the model constructor
nlay,nrow,ncol = 10,20,5
model_ws = os.path.join("data","external_demo")

if os.path.exists(model_ws):
    shutil.rmtree(model_ws)

# the place for all of your hand made and costly model inputs    
array_dir = os.path.join("data","array_dir")
if os.path.exists(array_dir):
    shutil.rmtree(array_dir)    
os.mkdir(array_dir)    

# lets make an external path relative to the model_ws
ml = flopy.modflow.Modflow(model_ws=model_ws, external_path="ref")

# explicitly reset the free_format flag BEFORE ANY PACKAGES ARE MADE!!!
ml.array_free_format = False

dis = flopy.modflow.ModflowDis(ml,nlay=nlay,nrow=nrow,ncol=ncol,steady=False,nper=2)

hk = np.zeros((nlay,nrow,ncol)) + 5.0
vka = np.zeros_like(hk)
fnames = []
for i,h in enumerate(hk):
    fname = os.path.join(array_dir,"hk_{0}.ref".format(i+1))
    fnames.append(fname)
    np.savetxt(fname,h)
    vka[i] = i+1
lpf = flopy.modflow.ModflowLpf(ml,hk=fnames,vka=vka)
ml.lpf.ss.how = "internal"
warmup_recharge = np.ones((nrow,ncol))
important_recharge = np.random.random((nrow,ncol))
fname = os.path.join(array_dir,"important_recharge.ref")
np.savetxt(fname,important_recharge)
rch = flopy.modflow.ModflowRch(ml,rech={0:warmup_recharge,1:fname})

ml.write_input()

Util2d:delr: resetting 'how' to external
Util2d:delc: resetting 'how' to external
Util2d:model_top: resetting 'how' to external
Util2d:botm layer 1: resetting 'how' to external
Util2d:botm layer 2: resetting 'how' to external
Util2d:botm layer 3: resetting 'how' to external
Util2d:botm layer 4: resetting 'how' to external
Util2d:botm layer 5: resetting 'how' to external
Util2d:botm layer 6: resetting 'how' to external
Util2d:botm layer 7: resetting 'how' to external
Util2d:botm layer 8: resetting 'how' to external
Util2d:botm layer 9: resetting 'how' to external
Util2d:botm layer 10: resetting 'how' to external
Util2d hk layer 1: can't be free format...resetting
Util2d:ss layer 1: resetting 'how' to external
Util2d hk layer 2: can't be free format...resetting
Util2d:ss layer 2: resetting 'how' to external
Util2d hk layer 3: can't be free format...resetting
Util2d:ss layer 3: resetting 'how' to external
Util2d hk layer 4: can't be free format...resetting
Util2d:ss layer 4: resetting 'ho

We see that now the external arrays are being handled through the name file.  Let's look at the name file

In [18]:
open(os.path.join(ml.model_ws,ml.name+".nam"),'r').readlines()

['# Name file for MODFLOW-2005, generated by Flopy version 3.2.6.\n',
 '#xul:0; yul:20; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2; length_multiplier:1.0 ;start_datetime:1-1-1970\n',
 'LIST               2  modflowtest.list\n',
 'DIS               11  modflowtest.dis \n',
 'LPF               15  modflowtest.lpf \n',
 'RCH               19  modflowtest.rch \n',
 'DATA            1001  ref/delr.ref\n',
 'DATA            1002  ref/delc.ref\n',
 'DATA            1003  ref/model_top.ref\n',
 'DATA            1004  ref/botm_layer_1.ref\n',
 'DATA            1005  ref/botm_layer_2.ref\n',
 'DATA            1006  ref/botm_layer_3.ref\n',
 'DATA            1007  ref/botm_layer_4.ref\n',
 'DATA            1008  ref/botm_layer_5.ref\n',
 'DATA            1009  ref/botm_layer_6.ref\n',
 'DATA            1010  ref/botm_layer_7.ref\n',
 'DATA            1011  ref/botm_layer_8.ref\n',
 'DATA            1012  ref/botm_layer_9.ref\n',
 'DATA            1013  ref/botm_layer_10.ref\n',

### "free" and "binary" format

In [19]:
ml.dis.botm[0].format.binary = True
ml.write_input()

Util2d:delr: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/delr.ref
Util2d:delc: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/delc.ref
Util2d:model_top: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/model_top.ref
Util2d:botm layer 1: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_1.ref
Util2d:botm layer 2: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_2.ref
Util2d:botm layer 3: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_3.ref
Util2d:botm layer 4: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_4.ref
Util2d:botm layer 5: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref

In [20]:
open(os.path.join(ml.model_ws,ml.name+".nam"),'r').readlines()

['# Name file for MODFLOW-2005, generated by Flopy version 3.2.6.\n',
 '#xul:0; yul:20; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2; length_multiplier:1.0 ;start_datetime:1-1-1970\n',
 'LIST               2  modflowtest.list\n',
 'DIS               11  modflowtest.dis \n',
 'LPF               15  modflowtest.lpf \n',
 'RCH               19  modflowtest.rch \n',
 'DATA            1046  ref/delr.ref\n',
 'DATA            1047  ref/delc.ref\n',
 'DATA            1048  ref/model_top.ref\n',
 'DATA(BINARY)    1049  ref/botm_layer_1.ref REPLACE\n',
 'DATA            1050  ref/botm_layer_2.ref\n',
 'DATA            1051  ref/botm_layer_3.ref\n',
 'DATA            1052  ref/botm_layer_4.ref\n',
 'DATA            1053  ref/botm_layer_5.ref\n',
 'DATA            1054  ref/botm_layer_6.ref\n',
 'DATA            1055  ref/botm_layer_7.ref\n',
 'DATA            1056  ref/botm_layer_8.ref\n',
 'DATA            1057  ref/botm_layer_9.ref\n',
 'DATA            1058  ref/botm_layer_10

In [21]:
open(os.path.join(ml.model_ws,ml.name+".dis"),'r').readlines()

['# DIS package for  MODFLOW-2005, generated by Flopy.\n',
 '        10        20         5         2         4         2\n',
 '  0  0  0  0  0  0  0  0  0  0\n',
 '      1046         1           (5E15.6)        -1 #delr\n',
 '      1047         1          (20E15.6)        -1 #delc\n',
 '      1048         1           (5E15.6)        -1 #model_top\n',
 '     -1049         1           (BINARY)        -1 #botm layer 1\n',
 '      1050         1           (5E15.6)        -1 #botm layer 2\n',
 '      1051         1           (5E15.6)        -1 #botm layer 3\n',
 '      1052         1           (5E15.6)        -1 #botm layer 4\n',
 '      1053         1           (5E15.6)        -1 #botm layer 5\n',
 '      1054         1           (5E15.6)        -1 #botm layer 6\n',
 '      1055         1           (5E15.6)        -1 #botm layer 7\n',
 '      1056         1           (5E15.6)        -1 #botm layer 8\n',
 '      1057         1           (5E15.6)        -1 #botm layer 9\n',
 '      1058    

### The ```.how``` attribute
```Util2d``` includes a ```.how``` attribute that gives finer grained control of how arrays will written

In [22]:
ml.lpf.hk[0].how

'external'

This will raise an error since our model does not support free format...

In [23]:
try:
    ml.lpf.hk[0].how = "openclose"
    ml.lpf.hk[0].how
    ml.write_input()
except Exception as e:
    print('\n', e, '\n')

Util2d:delr: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/delr.ref
Util2d:delc: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/delc.ref
Util2d:model_top: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/model_top.ref
Util2d:botm layer 1: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_1.ref
Util2d:botm layer 2: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_2.ref
Util2d:botm layer 3: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_3.ref
Util2d:botm layer 4: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_4.ref
Util2d:botm layer 5: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref

So let's reset hk layer 1 back to external...

In [24]:
ml.lpf.hk[0].how = "external"
ml.lpf.hk[0].how

'external'

In [25]:
ml.dis.top.how = "external"

In [26]:
ml.write_input()

Util2d:delr: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/delr.ref
Util2d:delc: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/delc.ref
BaseModel.add_external() warning: replacing existing filename ref/model_top.ref
Util2d:botm layer 1: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_1.ref
Util2d:botm layer 2: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_2.ref
Util2d:botm layer 3: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_3.ref
Util2d:botm layer 4: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_4.ref
Util2d:botm layer 5: resetting 'how' to external
BaseModel.add_external() warning: replacing existing filename ref/botm_layer_5.ref
Util2d:botm layer 6: resetti

In [27]:
open(os.path.join(ml.model_ws,ml.name+".dis"),'r').readlines()

['# DIS package for  MODFLOW-2005, generated by Flopy.\n',
 '        10        20         5         2         4         2\n',
 '  0  0  0  0  0  0  0  0  0  0\n',
 '      1104         1           (5E15.6)        -1 #delr\n',
 '      1105         1          (20E15.6)        -1 #delc\n',
 '      1106         1           (5E15.6)        -1 #model_top\n',
 '     -1107         1           (BINARY)        -1 #botm layer 1\n',
 '      1108         1           (5E15.6)        -1 #botm layer 2\n',
 '      1109         1           (5E15.6)        -1 #botm layer 3\n',
 '      1110         1           (5E15.6)        -1 #botm layer 4\n',
 '      1111         1           (5E15.6)        -1 #botm layer 5\n',
 '      1112         1           (5E15.6)        -1 #botm layer 6\n',
 '      1113         1           (5E15.6)        -1 #botm layer 7\n',
 '      1114         1           (5E15.6)        -1 #botm layer 8\n',
 '      1115         1           (5E15.6)        -1 #botm layer 9\n',
 '      1116    

In [28]:
open(os.path.join(ml.model_ws,ml.name+".lpf"),'r').readlines()

['# LPF package for  MODFLOW-2005, generated by Flopy.\n',
 '         0    -1E+30         0  \n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '         0         0         0         0         0         0         0         0         0         0\n',
 '      1117         1           (5E15.6)        -1 #hk layer 1\n',
 '      1118         1           (5E15.6)        -1 #vka layer 1\n',
 '      1119         1           (5E15.6)        -1 #ss layer 1\n',
 '      1120         1           (5E15.6)        -1 #hk layer 2\n',
 '      1121         1           (5E15.6)   

In [29]:
open(os.path.join(ml.model_ws,ml.name+".nam"),'r').readlines()

['# Name file for MODFLOW-2005, generated by Flopy version 3.2.6.\n',
 '#xul:0; yul:20; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2; length_multiplier:1.0 ;start_datetime:1-1-1970\n',
 'LIST               2  modflowtest.list\n',
 'DIS               11  modflowtest.dis \n',
 'LPF               15  modflowtest.lpf \n',
 'RCH               19  modflowtest.rch \n',
 'DATA            1104  ref/delr.ref\n',
 'DATA            1105  ref/delc.ref\n',
 'DATA            1106  ref/model_top.ref\n',
 'DATA(BINARY)    1107  ref/botm_layer_1.ref REPLACE\n',
 'DATA            1108  ref/botm_layer_2.ref\n',
 'DATA            1109  ref/botm_layer_3.ref\n',
 'DATA            1110  ref/botm_layer_4.ref\n',
 'DATA            1111  ref/botm_layer_5.ref\n',
 'DATA            1112  ref/botm_layer_6.ref\n',
 'DATA            1113  ref/botm_layer_7.ref\n',
 'DATA            1114  ref/botm_layer_8.ref\n',
 'DATA            1115  ref/botm_layer_9.ref\n',
 'DATA            1116  ref/botm_layer_10